In [3]:
import os, json
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

def vertical_to_horizontal(filtered_df):
    naming_df = filtered_df.pivot(index='y', columns=['x'], values="text")
    return(naming_df)

def cleaning(naming_df):
    naming_df['sold_by'] = naming_df['sold_by'].str.split("Opens").str[0]
    naming_df['total_price'] = naming_df['total_price'].str.split("Item").str[0]
    return(naming_df)

def main():
    path_to_json = 'E:/growbydata/Growbydata/first_task_files/'
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    final_json_to_csv_df = pd.DataFrame()
    from_file = 1
    index_file_mapping={}

    for file_name in (json_files):
        #print(file_name)
        data = json.load(open(f"./first_task_files/{file_name}"))
        df = pd.json_normalize(data["data"])
        df['file_from'] = file_name
        selected_all_row_df = df[df['element'].str.contains('TD') & df['attributes.class'].str.contains('SH30Lb')]
        filtered_df = selected_all_row_df[['text', 'y', 'x', 'file_from']]        
        filtered_df = filtered_df[filtered_df['x'] != 0]  # to drop rows whose x value is 0 for 80 file

        if len(filtered_df)==0: # to remove pivot error 
            mydict = {'text': [], 'y': [0,0,0,0,0], 'x': [39.0, 287.0, 569.0, 699.0, 807.0]}
            filtered_df = pd.DataFrame({ key:pd.Series(value) for key, value in mydict.items() }) 
            
        naming_df = vertical_to_horizontal(filtered_df)
        naming_df.columns = ["sold_by", "details", "item_price", "total_price", "file_name"]
        
        if naming_df.any(axis=0).any() == True:  # check any data present or not in 36 file
            naming_df = cleaning(naming_df)
        
        del naming_df["file_name"]
        indexx = []
        field = []
        value = []
        j = 1
        
        for index, content in naming_df.iterrows():            
            for i in range(len(content)):
                indexx.append(f" {from_file}.{j}.{i + 1}")                
            j = j + 1
         
        file_name = file_name.split('.') # for file number
        file_name=file_name[0]
        index_file_mapping[file_name] = from_file   # for mapping     
        
        j = 0
        for index, content in naming_df.iterrows():
            for i in range(len(content)):
                field.append(content.index[i])
                value.append(content[i])
                j = j + 1
        from_file = from_file + 1
        
        # series
        s1 = pd.Series(indexx)
        s2 = pd.Series(field)
        s3 = pd.Series(value)
        final_df = pd.concat([s1, s2, s3], axis=1)
        final_df.columns = ['index', 'field', 'value']
        final_df = final_df.sort_values(by=['field', 'index'])
        final_df = final_df.fillna("Na")
        final_json_to_csv_df = pd.concat([final_json_to_csv_df, final_df])
    
    index_file_mapping_df = pd.DataFrame(list(index_file_mapping.items()), columns = ['file_name','Indexing'])
    print(index_file_mapping_df)
    #index_file_mapping_df.to_csv("map_new.csv")
    #print(final_json_to_csv_df)
   # final_json_to_csv_df.to_csv("final_3333_1222.csv",index=False)
if __name__ == "__main__":
    main()

   file_name  Indexing
0         10         1
1         11         2
2         12         3
3         13         4
4         14         5
5          2         6
6         32         7
7         33         8
8         34         9
9         36        10
10        37        11
11        38        12
12        39        13
13         4        14
14        40        15
15        41        16
16        42        17
17        43        18
18        44        19
19         6        20
20         7        21
21        77        22
22        78        23
23        80        24
24        82        25
25        83        26
26        84        27
27        85        28
28        86        29
29         9        30
